# Quadrant analysis (from raw data)

In [ ]:
import pandas as pd
import numpy as np
import glob

from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import date_format

In [ ]:
# Data location
project_path = './'
data_path = project_path + '../../data/'

# Input path
input_path = project_path + '../data/raw_truck/'

# Output path
graphs_path = project_path + '../graphs/'

In [ ]:
def load_all_raw(directory, silent=False):
    if (not silent):
        print('  - Raw EC files:')
        
    file_list = sorted(glob.glob(directory + '**/*.csv', recursive=True))
    data_list = []
    for idx, fn in enumerate(file_list):
        print('    -', fn.split('\\')[-1])
        # Read file
        temp = load_raw_sonic(fn)
        # Append to list of dfs
        data_list.append(temp)
    # Combine all the read data
    df = pd.concat(data_list, axis=0, ignore_index=True)
    return(df)

def load_raw_sonic(fn):
    temp = pd.read_csv(fn, skiprows=1)
    temp['DateTime'] = pd.to_datetime(temp['date'] + ' ' + temp['time'], format='%d/%m/%y %H:%M:%S.%f', utc=False)
    # Remove unnecessary columns
    temp.drop(['date','time'], axis=1, inplace=True)
    # Move timestamp to front
    col = temp.pop('DateTime')
    temp.insert(0, col.name, col, allow_duplicates=True)
    # Rename columns
    temp.rename({'sonic T': 'Ts'}, axis=1, inplace=True)
    temp.rename({' "CO2 ppm"': 'co2_umol_mol'}, axis=1, inplace=True)
    temp.rename({' "H2O mmol/mol"': 'h2o_mmol_mol'}, axis=1, inplace=True)
    temp.rename({' "pressure Pa"': 'P_Pa'}, axis=1, inplace=True)
    return(temp)

def add_ecosystem(df):
    df = df.copy()
    # Add ecosystem
    df['Ecosystem'] = np.nan
    df.loc[df['DateTime'] < '2019-07-16', 'Ecosystem'] = 'Desert background'
    df.loc[df['DateTime'] >= '2019-07-16', 'Ecosystem'] = 'PV field'
    # Create day identifier
    df['day'] = df['DateTime'].dt.strftime('%Y-%m-%d')
    # shift column 'timestamp' to first position
    col = df.pop('day')
    df.insert(0, col.name, col, allow_duplicates=True)
    col = df.pop('Ecosystem')
    df.insert(0, col.name, col, allow_duplicates=True)
    col = df.pop('DateTime')
    df.insert(0, col.name, col, allow_duplicates=True)
    return(df)

In [ ]:
def calculate_deviation(vect):
    dev = vect - np.mean(vect)
    return(dev)

def calculate_tke(u, v, w): # [m2 s-2]
    dev_u = calculate_deviation(u)
    dev_v = calculate_deviation(v)
    dev_w = calculate_deviation(w)
    
    tke_out = (1/2) * ( np.mean(dev_u**2) + np.mean(dev_v**2) + np.mean(dev_w**2) )
    return(tke_out)

def calculate_cov(a, b):
    dev_a = calculate_deviation(a)
    dev_b = calculate_deviation(b)
  
    out = np.mean(dev_a * dev_b)
    return(out)

def calculate_uStar(u, v, w):
    dev_u = calculate_deviation(u)
    dev_v = calculate_deviation(v)
    dev_w = calculate_deviation(w)
  
    uStar_out = ((np.mean(dev_u * dev_w))**2 + (np.mean(dev_v * dev_w))**2)**(1/4)
    return(uStar_out)

def calculate_mechShear_uStar(u, v, w, uStar, z):
    k = 0.4 # von Karman constant
    dev_u = calculate_deviation((u**2 + v**2)**(1/2)) # total horizontal wind, no v component
    dev_w = calculate_deviation(w)
  
    dudz = uStar / (k*z)
  
    mechShear_out = -np.mean(dev_u * dev_w) * dudz
    return(mechShear_out)

# mean of w' * ws' (horizontal windspeed)
def calculate_cov_uw(u,v,w):
    cov_wu = calculate_cov((u**2 + v**2)**(1/2), w)
    return(cov_wu)

def calculate_w_prime(w):
    # w'
    dev_w = calculate_deviation(w)
    return(dev_w)

def calculate_horizontal_windspeed(u,v):
    # horizontal wind speed u
    ws_horizontal = (u**2 + v**2)**(1/2)
    return(ws_horizontal)

def calculate_total_windspeed(u,v,w):
	# total windspeed in x, y & z direction
    ws_total = (u**2 + v**2 + w**2)**(1/2)
    return(ws_total)

In [ ]:
print('Loading data...')
df = load_all_raw(input_path)
df = add_ecosystem(df)

print('  - Calculating w\' and T\'')
df['w_prime'] = calculate_deviation(df['w'])
df['T_prime'] = calculate_deviation(df['Ts'])

print('Done...')

In [ ]:
# Calculate n per quadrant
def quadrant_count(temp):
    temp = temp.copy()
    
    # Creat quadrants
    temp['quadrant'] = np.nan
    temp.loc[(temp['w_prime'] > 0) & (temp['T_prime'] < 0), 'quadrant'] = 'w\'>0, T\'<0'
    temp.loc[(temp['w_prime'] < 0) & (temp['T_prime'] < 0), 'quadrant'] = 'w\'<0, T\'<0'
    temp.loc[(temp['w_prime'] > 0) & (temp['T_prime'] > 0), 'quadrant'] = 'w\'>0, T\'>0'
    temp.loc[(temp['w_prime'] < 0) & (temp['T_prime'] > 0), 'quadrant'] = 'w\'<0, T\'>0'

    count_df = temp[['Ecosystem','quadrant','DateTime']].groupby(['Ecosystem','quadrant']).count()
    count_df.reset_index(inplace=True)
    count_df.rename({'DateTime': 'n'}, axis=1, inplace=True)
    return(count_df)

count_df = quadrant_count(df)
display(count_df)

# Assign coordinates for the text label in the figure
count_df['x'] = np.nan
count_df['y'] = np.nan

count_df.loc[count_df['quadrant'] == 'w\'>0, T\'<0', 'x'] = 2
count_df.loc[count_df['quadrant'] == 'w\'>0, T\'<0', 'y'] = -2

count_df.loc[count_df['quadrant'] == 'w\'<0, T\'<0', 'x'] = -2
count_df.loc[count_df['quadrant'] == 'w\'<0, T\'<0', 'y'] = -2

count_df.loc[count_df['quadrant'] == 'w\'>0, T\'>0', 'x'] = 2
count_df.loc[count_df['quadrant'] == 'w\'>0, T\'>0', 'y'] = 4

count_df.loc[count_df['quadrant'] == 'w\'<0, T\'>0', 'x'] = -2
count_df.loc[count_df['quadrant'] == 'w\'<0, T\'>0', 'y'] = 4

count_df['n_label'] = 'n=' + count_df['n'].astype(str)

In [ ]:
plt = ggplot(df)
plt = plt + geom_point(aes(x='w_prime', y='T_prime'), colour='#939393', size=0.05)
plt = plt + facet_wrap('Ecosystem')
plt = plt + labs(x='w\' [$m~s^{-1}$]', y='T\' [$K$]')
plt = plt + geom_hline(yintercept = 0, colour='black')
plt = plt + geom_vline(xintercept = 0, colour='black')
plt = plt + theme_bw()
plt = plt + scale_x_continuous(breaks=range(-5, 5, 1))
plt = plt + scale_y_continuous(breaks=range(-5, 5, 1))
plt = plt + geom_text(data=count_df, mapping=aes(x='x', y='y', label='n_label'))
print(plt)

plt.save(graphs_path + 'quadrant_analysis.png', width=170, height=85, units='mm', dpi=600)
plt.save(graphs_path + 'quadrant_analysis.pdf', width=170, height=85, units='mm', dpi=600)